In [ ]:
import pyart
from pyart.io.sigmet import SigmetFile
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import inspect
import configparser
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from datetime import datetime
import utils
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Set some matplotlib style params
background_color = "white"
emph_color = "black"
plt.rcParams.update({
    "axes.facecolor": background_color,
    "text.usetex": False,
    "text.latex.preamble": r'',
    "figure.titleweight": "bold",
    "axes.titleweight": "bold",
    "axes.labelweight": "bold",
# #     "font.family": "sans-serif",
# #     "font.sans-serif": ["Helvetica"]
# #     'font.size': 12, 
# #     'font.family': 'Times New Roman',
# #     'mathtext.fontset': 'cm',
})

# Make sure correct pyart config is loaded
pyart.load_config(os.environ.get('PYART_CONFIG'))

In [ ]:
def add_estimated_SNR(fn, radar):
    sigmet_file = SigmetFile(fn)

    # Get reflectivity calibration value
    NEZ = (sigmet_file.ingest_header["task_configuration"]
                    ["task_calib_info"]["reflectivity_calibration"] / 16)
    # Ranges in the same shape as data
    R = np.tile(radar.range["data"], (radar.nrays, 1)) * 1e-3
    # Estimated SNR
    SNR = radar.fields["reflectivity"]["data"] - NEZ - 20 * np.log10(R)

    # Add a field
    radar.add_field_like('signal_to_noise_ratio', "estimated_signal_to_noise_ratio", SNR)

    sigmet_file.close()

In [ ]:
# Give path to file
fn = "/mnt/data/MWSA/Mar09_2021/WND-03/WRS210309151307.RAWZ7YM"
# Can also read some HDF5 files, if the number of range bins doesn't change!
# So it might be easiest to copy only one elevation to the file
# fn = "202005192245_radar.polar.fikuo.dataset1.h5"

# For ODIM HDF5
# radar = pyart.aux_io.read_odim_h5(fn)
# For RAW files
radar = pyart.io.read_sigmet(fn)
# Get start time of the radar scan
time = datetime.strptime(radar.time["units"], 'seconds since %Y-%m-%dT%H:%M:%SZ')

# Add estimated SNR as new field, similar examples work also for other new fields
add_estimated_SNR(fn, radar)
# Print the existing keys
print(radar.fields.keys())

In [ ]:
def plot_fourpanel_fig(radar, qtys, titles, rmax=100):
    cbar_ax_kws = {
        "width": "5%",  # width = 5% of parent_bbox width
        "height": "100%", 
        "loc": 'lower left',
        "bbox_to_anchor": (1.01, 0., 1, 1),
        "borderpad": 0,
    }

    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 10), sharex='col', sharey='row')
    display = pyart.graph.RadarDisplay(radar)
    fmt = mpl.ticker.StrMethodFormatter("{x:.0f}")
    for ax, title, qty in zip(axes.flat, titles, qtys):
        # Create inset axis for colorbar
        cax = inset_axes(ax, bbox_transform=ax.transAxes, **cbar_ax_kws)
        # Get norm and colormap for the quantity
        cmap, norm = utils.get_colormap(qty)
        if norm is None:
            norm = mpl.colors.Normalize(vmin=utils.QTY_RANGES[qty][0], vmax=utils.QTY_RANGES[qty][1])

        # Plot the radar image
        display.plot(
            utils.PYART_FIELDS[qty], 0, title='',
            vmin=utils.QTY_RANGES[qty][0], vmax=utils.QTY_RANGES[qty][1], 
            ax=ax, axislabels_flag=False, colorbar_flag=False,
            cmap=cmap, norm=norm, zorder=10,
        )

        # Add colorbar to axis we created before
        cbar = plt.colorbar(
            mpl.cm.ScalarMappable(norm=norm, cmap=cmap), 
            format=mpl.ticker.StrMethodFormatter(utils.QTY_FORMATS[qty]),
            orientation='vertical', cax=cax, ax=None)
        cbar.set_label(label=utils.COLORBAR_TITLES[qty], weight='bold')

        # Add some range rings, add more to the list as you wish
        display.plot_range_rings([250], ax=ax, lw=0.5, col=emph_color)
        # Could also add grid lines
        # display.plot_grid_lines(ax=ax, col="grey", ls=":")
        ax.set_title(title, y=-0.12)


    # x-axis setup
    for ax in axes[1][:].flat:
        ax.set_xlabel("Distance from radar (km)")
        ax.set_title(ax.get_title(), y=-0.22)
        ax.xaxis.set_major_formatter(fmt)

    # y-axis setup
    for ax in axes.flat[::2]:
        ax.set_ylabel("Distance from radar (km)")
        ax.yaxis.set_major_formatter(fmt)

    # In all axis, set up rax range and aspect
    for ax in axes.flat:
        ax.set_xlim([-rmax, rmax])
        ax.set_ylim([-rmax, rmax])
        ax.set_aspect(1)
        ax.grid(zorder=0, linestyle='-', linewidth=0.3)

    # title = display.generate_title(field="reflectivity", sweep=0).split("\n")[0]
    fig.suptitle(f"{time:%Y/%m/%d %H:%M} UTC", y=0.92)

    # Some adjusting to make plot a bit tighter, adjust as needed
    fig.subplots_adjust(wspace=0, hspace=0.2)
    fig.savefig(f"radar_vars_{time:%Y%m%d%H%M}_{radar.fixed_angle['data'][0]:.1f}.png", dpi=600, bbox_inches="tight")

In [ ]:
titles = [
    r"(a) $Z_{e}$",
    r"(b) $v$",
    r"(c) Zdr",
    r"(d) Kdp"
]
qtys = [
    "DBZH",
    "VRAD",
    "ZDR",
    "KDP",
]

plot_fourpanel_fig(radar, qtys, titles)
